In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '' 
import jax

jax.config.update('jax_enable_x64', True)
jax.config.update("jax_default_matmul_precision", "highest")

from mace.calculators import mace_mp,mace_off
from ase import build
from fennol import FENNIX
import yaml
import jax.numpy as jnp
import numpy as np  
from fennol.ase import FENNIXCalculator
from fennol.utils import AtomicUnits as au
from copy import deepcopy
from mace import modules as torch_modules
import e3nn as e3nn_torch
import torch
from fennol.utils.activations import normalize_activation

from ase.units import Hartree
print(au.EV)
print(Hartree)

e3nn_torch.__version__

27.211386024367243
27.211386024367243


'0.4.4'

In [2]:
atoms = build.molecule('H2O')
model_name = "mace_mp_large"
atoms.set_positions([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device='cpu')
atoms.calc = calc
# print(atoms.get_potential_energy())

torch_model_mp = calc.models[0].cpu()
species_order = torch_model_mp.atomic_numbers.tolist()
print("species_order:",species_order)
print(max(species_order))
num_species = len(species_order)
print(num_species)

scale = torch_model_mp.scale_shift.scale.item()
shift = torch_model_mp.scale_shift.shift.item()
print("scale",scale/au.EV)
print("shift",shift/au.EV)

avg_num_neigh = torch_model_mp.interactions[0].avg_num_neighbors
print("avg_num_neighbors",avg_num_neigh)
print(torch_model_mp)



Using Materials Project MACE for MACECalculator with /home/pthomas/.cache/mace/5f5yavf3
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
species_order: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 89, 90, 91, 92, 93, 94]
94
89
scale 0.02955895460120917
shift 0.006055514143288505
avg_num_neighbors 25.57754135131836
ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 256x0e | 22784 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=4.5, num_basis=8, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=4.5)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_

In [3]:
r_max = torch_model_mp.radial_embedding.bessel_fn.r_max.item()
num_basis = torch_model_mp.radial_embedding.bessel_fn.bessel_weights.numel()
p = torch_model_mp.radial_embedding.cutoff_fn.p.item()
max_ell = torch_model_mp.spherical_harmonics._lmax
num_inter = len(torch_model_mp.products)
corr = len(torch_model_mp.products[0].symmetric_contractions.contractions[0].weights)+1
print(torch_model_mp.readouts[1].non_linearity.acts[0].cst)
print(f"{r_max=},{num_basis=},{p=},{max_ell=},{num_inter=},{num_species=},{avg_num_neigh=},{corr=}")
torch_model = torch_modules.ScaleShiftMACE(
    atomic_inter_scale=scale,
    atomic_inter_shift=shift,
    r_max=r_max,
    num_bessel=num_basis,
    num_polynomial_cutoff=p,
    max_ell=max_ell,
    interaction_cls_first=torch_modules.RealAgnosticResidualInteractionBlock,
    interaction_cls=torch_modules.RealAgnosticResidualInteractionBlock,
    num_interactions=num_inter,
    num_elements=num_species,
    hidden_irreps=e3nn_torch.o3.Irreps("256x0e+256x1o"),
    MLP_irreps="16x0e",
    avg_num_neighbors=avg_num_neigh,
    correlation=corr,
    atomic_energies=torch_model_mp.atomic_energies_fn.atomic_energies.clone().detach(),
    atomic_numbers=torch_model_mp.atomic_numbers.clone().detach(),
    gate=torch.nn.SiLU(),
).double()
print(torch_model)

1.6791767923989418
r_max=4.5,num_basis=8,p=5.0,max_ell=3,num_inter=2,num_species=89,avg_num_neigh=25.57754135131836,corr=3


/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/models.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "atomic_numbers", torch.tensor(atomic_numbers, dtype=torch.int64)
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/mace/modules/blocks.py:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(atomic_

ScaleShiftMACE(
  (node_embedding): LinearNodeEmbeddingBlock(
    (linear): Linear(89x0e -> 256x0e | 22784 weights)
  )
  (radial_embedding): RadialEmbeddingBlock(
    (bessel_fn): BesselBasis(r_max=4.5, num_basis=8, trainable=False)
    (cutoff_fn): PolynomialCutoff(p=5.0, r_max=4.5)
  )
  (spherical_harmonics): SphericalHarmonics()
  (atomic_energies_fn): AtomicEnergiesBlock(energies=[-3.6674, -1.3220, -3.4782, -4.7250, -7.7279, -8.4045, -7.3594, -7.2844, -4.8962, -0.0301, -2.7527, -2.8138, -4.8444, -7.6966, -6.9632, -4.6738, -2.8140, -0.0623, -2.6252, -5.3848, -7.9162, -10.2708, -8.6724, -9.2372, -8.3056, -7.0547, -5.5709, -5.1741, -3.2510, -1.2880, -3.5322, -4.7113, -3.9718, -3.8862, -2.5183, 6.7068, -2.5661, -4.9390, -10.1536, -11.8571, -12.1416, -8.7861, -8.7954, -7.7784, -6.8387, -4.8812, -2.0606, -0.6440, -2.7886, -3.8168, -3.5880, -2.8806, -1.6361, 9.7960, -2.7486, -4.9867, -8.9295, -8.7417, -8.0122, -8.2336, -7.6245, -8.1614, -13.5915, -18.5381, -7.6446, -8.1150, -7.6146, -6.

/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/pthomas/miniconda3/envs/mace/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.w

In [4]:
torch_model.load_state_dict(torch_model_mp.state_dict())
calc = mace_mp(model="large", dispersion=False, default_dtype="float64", device='cpu')
calc.models[0] = torch_model
atoms.calc = calc
print(calc.get_potential_energy(atoms,force_consistent=True))

Using Materials Project MACE for MACECalculator with /home/pthomas/.cache/mace/5f5yavf3
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
-14.017221861246675


In [5]:
print(torch_model.products[0].symmetric_contractions.contractions[0].correlation)


3


In [6]:
model_params = yaml.load(open(model_name+".yaml"), Loader=yaml.FullLoader)
model_fennol = FENNIX(**model_params,rng_key=jax.random.PRNGKey(1)) 

In [7]:
variables = deepcopy(model_fennol.variables)
nn_energy_params = variables["params"]["nn_energy"]

for k in nn_energy_params.keys():
    print(k)
    if isinstance(nn_energy_params[k],dict):
        for kk in nn_energy_params[k].keys():
            print("   ",kk)


SymmetricContraction_0
    w1_0e
    w1_1o
    w2_0e
    w2_1o
    w3_0e
    w3_1o
SymmetricContraction_1
    w1_0e
    w2_0e
    w3_0e
hidden_linear_readout_last
    w[0,0] 256x0e,16x0e
linear_contraction_0
    w[0,0] 256x0e,256x0e
    w[1,1] 256x1o,256x1o
linear_contraction_1
    w[0,0] 256x0e,256x0e
linear_dn_0
    w[0,0] 256x0e,256x0e
    w[1,1] 256x1o,256x1o
    w[2,2] 256x2e,256x2e
    w[3,3] 256x3o,256x3o
linear_dn_1
    w[0,0] 512x0e,256x0e
    w[1,1] 768x1o,256x1o
    w[2,2] 768x2e,256x2e
    w[3,3] 512x3o,256x3o
linear_readout_0
    w[0,0] 256x0e,1x0e
linear_readout_last
    w[0,0] 16x0e,1x0e
linear_up_0
    w[0,0] 256x0e,256x0e
linear_up_1
    w[0,0] 256x0e,256x0e
    w[1,1] 256x1o,256x1o
radial_network_0
    Dense_0
    Dense_1
    Dense_2
    Dense_3
radial_network_1
    Dense_0
    Dense_1
    Dense_2
    Dense_3
skip_tp_0
    w[0,0] 256x0e,256x0e
skip_tp_1
    w[0,0] 256x0e,256x0e
species_encoding


In [8]:
def linear_torch_to_jax(linear):
    return {
        f"w[{ins.i_in},{ins.i_out}] {linear.irreps_in[ins.i_in]},{linear.irreps_out[ins.i_out]}": jnp.asarray(
            w.data
        )
        for i, ins, w in linear.weight_views(yield_instruction=True)
    }

def skip_tp_torch_to_jax(tp):
    return {
        f"w[{ins.i_in1},{ins.i_out}] {tp.irreps_in1[ins.i_in1]},{tp.irreps_out[ins.i_out]}": jnp.moveaxis(
            jnp.asarray(w.data), 1, 0
        )/num_species**0.5
        for i, ins, w in tp.weight_views(yield_instruction=True)
    }

In [9]:
## get initial species encodings
species_encodings = np.zeros(nn_energy_params["species_encoding"].shape,dtype=np.float64)
species_encodings_torch = torch_model.node_embedding.linear.weight.detach().numpy().reshape(num_species,-1)
assert species_encodings.shape[1] == species_encodings_torch.shape[1]
factor = num_species**-0.5
for i,z in enumerate(species_order):
    species_encodings[z,:] = species_encodings_torch[i,:]*factor
    # print(f"{z}: {species_encodings[z]}")
            
nn_energy_params["species_encoding"] = jnp.array(species_encodings,dtype=jnp.float64)
print(species_encodings[1,:10])
print(species_encodings_torch[0,:10])
print(torch_model.node_embedding.linear.instructions)

[ 0.05964225  0.03127083 -0.01582405 -0.03060492  0.0286575  -0.01841103
 -0.22729727 -0.17191443  0.00620572  0.08738091]
[ 0.56266386  0.29500845 -0.14928375 -0.28872619  0.27035428 -0.17368927
 -2.14431819 -1.62183749  0.05854462  0.82434988]
[Instruction(i_in=0, i_out=0, path_shape=(89, 256), path_weight=0.105999788000636)]


In [10]:
# species energy shift

print(len(model_fennol.variables["params"]["energy_shift"]["constant"]))
print(len(torch_model.atomic_energies_fn.atomic_energies))
energy_shift = np.zeros(len(model_fennol.variables["params"]["energy_shift"]["constant"]),dtype=np.float64)
for i,z in enumerate(species_order):
    # print(f"{z}: {model_fennol.variables['params']['energy_shift']['constant'][z]} {torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV}")
    energy_shift[z] = torch_model.atomic_energies_fn.atomic_energies[i].data/au.EV
variables["params"]["energy_shift"]["constant"] = jnp.array(energy_shift,dtype=jnp.float64)
print(variables["params"]["energy_shift"]["constant"])
    

119
89
[ 0.         -0.13477364 -0.04858182 -0.12781999 -0.17363932 -0.28399661
 -0.30886129 -0.27045406 -0.26769809 -0.17993064 -0.00110461 -0.10116055
 -0.10340619 -0.17802731 -0.28284597 -0.25589336 -0.17175941 -0.10341201
 -0.00229076 -0.09647349 -0.19788743 -0.2909149  -0.37744571 -0.31870447
 -0.3394593  -0.30522592 -0.25925441 -0.20472737 -0.19014643 -0.11947335
 -0.04733186 -0.12980535 -0.17313813 -0.14596006 -0.14281601 -0.09254634
  0.24647191 -0.09430087 -0.18150468 -0.37313806 -0.435739   -0.44619728
 -0.32288302 -0.32322626 -0.28584915 -0.25131857 -0.1793814  -0.07572414
 -0.02366622 -0.10247825 -0.1402652  -0.13185597 -0.1058598  -0.06012382
  0.3599964  -0.10101055 -0.18325677 -0.32815222 -0.3212518  -0.29444187
 -0.30257916 -0.28019682 -0.29992475 -0.49947663 -0.68126396 -0.28093213
 -0.29822258 -0.2798303  -0.25098414 -0.28738069 -0.13145413 -0.27448824
 -0.46931268 -0.51856921 -0.34376807 -0.41807717 -0.35322028 -0.26905146
 -0.19492964 -0.0877323   0.00911094 -0.0850

In [11]:
# skip tp 0
for layer in range(2):
    print(f'skip_tp_{layer}')
    skip_params_torch = skip_tp_torch_to_jax(
        torch_model.interactions[layer].skip_tp
    )
    print(skip_params_torch.keys())
    print(nn_energy_params[f"skip_tp_{layer}"].keys())
    for k in nn_energy_params[f'skip_tp_{layer}'].keys():
        print(skip_params_torch[k].shape)
        print(nn_energy_params[f'skip_tp_{layer}'][k].shape)
        skip_params = np.zeros_like(nn_energy_params[f'skip_tp_{layer}'][k])
        assert skip_params.shape[1:] == nn_energy_params[f'skip_tp_{layer}'][k].shape[1:]
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = np.array(skip_params_torch[k][i,:,:])

        nn_energy_params[f'skip_tp_{layer}'][k] = jnp.array(skip_params,dtype=jnp.float64)

# # skip tp 1
# print()
# print("skip tp 1")
# skip_params_torch = skip_tp_torch_to_jax(
#         torch_model.interactions[1].skip_tp
#     )
# print(skip_params_torch.keys())
# print(nn_energy_params["skip_tp_1"].keys())

# print(skip_params_torch['w[0,0] 128x0e,128x0e'].shape)
# print(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape)
# skip_params = np.zeros_like(nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'])
# assert skip_params.shape[1:] == nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'].shape[1:]
# for i,z in enumerate(species_order):
#     skip_params[z,:,:] = np.array(skip_params_torch['w[0,0] 128x0e,128x0e'][i,:,:])

# nn_energy_params["skip_tp_1"]['w[0,0] 128x0e,128x0e'] = jnp.array(skip_params,dtype=jnp.float64)

skip_tp_0
dict_keys(['w[0,0] 256x0e,256x0e'])
dict_keys(['w[0,0] 256x0e,256x0e'])
(89, 256, 256)
(120, 256, 256)
skip_tp_1
dict_keys(['w[0,0] 256x0e,256x0e'])
dict_keys(['w[0,0] 256x0e,256x0e'])
(89, 256, 256)
(120, 256, 256)


In [12]:
# linear up
for i in range(2):
    print()
    print(f"linear_up_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear_up)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_up_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_up_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_up_{i}"][k].shape
        nn_energy_params[f"linear_up_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# linear down
for i in range(2):
    print()
    print(f"linear_dn_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.interactions[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_dn_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        print(nn_energy_params[f"linear_dn_{i}"][k].shape)
        assert up_params_torch[k].shape == nn_energy_params[f"linear_dn_{i}"][k].shape
        nn_energy_params[f"linear_dn_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

# contraction linear
for i in range(2):
    print()
    print(f"linear_contraction_{i}")
    up_params_torch = linear_torch_to_jax(torch_model.products[i].linear)
    print(up_params_torch.keys())
    print(nn_energy_params[f"linear_contraction_{i}"].keys())
    for k in up_params_torch.keys():
        print(k)
        print(up_params_torch[k].shape)
        try:
            print(nn_energy_params[f"linear_contraction_{i}"][k].shape)
            assert up_params_torch[k].shape == nn_energy_params[f"linear_contraction_{i}"][k].shape
            nn_energy_params[f"linear_contraction_{i}"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)
        except KeyError:
            print(f"key {k} not found in nn_energy_params[f'linear_contraction_{i}']")



linear_up_0
dict_keys(['w[0,0] 256x0e,256x0e'])
dict_keys(['w[0,0] 256x0e,256x0e'])
w[0,0] 256x0e,256x0e
(256, 256)
(256, 256)

linear_up_1
dict_keys(['w[0,0] 256x0e,256x0e', 'w[1,1] 256x1o,256x1o'])
dict_keys(['w[0,0] 256x0e,256x0e', 'w[1,1] 256x1o,256x1o'])
w[0,0] 256x0e,256x0e
(256, 256)
(256, 256)
w[1,1] 256x1o,256x1o
(256, 256)
(256, 256)

linear_dn_0
dict_keys(['w[0,0] 256x0e,256x0e', 'w[1,1] 256x1o,256x1o', 'w[2,2] 256x2e,256x2e', 'w[3,3] 256x3o,256x3o'])
dict_keys(['w[0,0] 256x0e,256x0e', 'w[1,1] 256x1o,256x1o', 'w[2,2] 256x2e,256x2e', 'w[3,3] 256x3o,256x3o'])
w[0,0] 256x0e,256x0e
(256, 256)
(256, 256)
w[1,1] 256x1o,256x1o
(256, 256)
(256, 256)
w[2,2] 256x2e,256x2e
(256, 256)
(256, 256)
w[3,3] 256x3o,256x3o
(256, 256)
(256, 256)

linear_dn_1
dict_keys(['w[0,0] 512x0e,256x0e', 'w[1,1] 768x1o,256x1o', 'w[2,2] 768x2e,256x2e', 'w[3,3] 512x3o,256x3o'])
dict_keys(['w[0,0] 512x0e,256x0e', 'w[1,1] 768x1o,256x1o', 'w[2,2] 768x2e,256x2e', 'w[3,3] 512x3o,256x3o'])
w[0,0] 512x0e,256x0e
(5

In [13]:
# readout block
print(f"linear_readout_0")
up_params_torch = linear_torch_to_jax(torch_model.readouts[0].linear)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_0"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_0"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_0"][k].shape
    nn_energy_params[f"linear_readout_0"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)

print()
print(f"hidden_linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_1)
print(up_params_torch.keys())
print(nn_energy_params[f"hidden_linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"hidden_linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"hidden_linear_readout_last"][k].shape
    nn_energy_params[f"hidden_linear_readout_last"][k] = jnp.array(up_params_torch[k],dtype=jnp.float64)


f,silu_norm = normalize_activation(jax.nn.silu,return_scale=True)
print("silu_norm",silu_norm,1.6765647703084872)
cst = torch_model.readouts[1].non_linearity.acts[0].cst / silu_norm

print()
print(f"linear_readout_last")
up_params_torch = linear_torch_to_jax(torch_model.readouts[1].linear_2)
print(up_params_torch.keys())
print(nn_energy_params[f"linear_readout_last"].keys())
for k in up_params_torch.keys():
    print(k)
    print(up_params_torch[k].shape)
    print(nn_energy_params[f"linear_readout_last"][k].shape)
    assert up_params_torch[k].shape == nn_energy_params[f"linear_readout_last"][k].shape
    nn_energy_params[f"linear_readout_last"][k] = cst*jnp.array(up_params_torch[k],dtype=jnp.float64)

print(torch_model.readouts[1].non_linearity.acts[0].cst)


linear_readout_0
dict_keys(['w[0,0] 256x0e,1x0e'])
dict_keys(['w[0,0] 256x0e,1x0e'])
w[0,0] 256x0e,1x0e
(256, 1)
(256, 1)

hidden_linear_readout_last
dict_keys(['w[0,0] 256x0e,16x0e'])
dict_keys(['w[0,0] 256x0e,16x0e'])
w[0,0] 256x0e,16x0e
(256, 16)
(256, 16)
silu_norm 1.6765620560798837 1.6765647703084872

linear_readout_last
dict_keys(['w[0,0] 16x0e,1x0e'])
dict_keys(['w[0,0] 16x0e,1x0e'])
w[0,0] 16x0e,1x0e
(16, 1)
(16, 1)
1.6791767923989418


In [14]:
# radial net

for i in range(2):
    print("radial net",i)
    print(nn_energy_params[f"radial_network_{i}"].keys())
    w  = torch_model.interactions[i].conv_tp_weights.layer0.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_0"]["kernel"] = jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer1.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_1"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer2.weight.detach().numpy()
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape)
    w = w/w.shape[0]**0.5
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_2"]["kernel"] =  jnp.array(w,dtype=jnp.float64)

    cst = torch_model.interactions[i].conv_tp_weights.layer0.act.cst/ silu_norm
    print("cst",cst)

    w  = cst*torch_model.interactions[i].conv_tp_weights.layer3.weight.detach().numpy()
    w = w/w.shape[0]**0.5
    print(w.shape,nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape)
    assert w.shape == nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"].shape
    nn_energy_params[f"radial_network_{i}"]["Dense_3"]["kernel"] =  jnp.array(w,dtype=jnp.float64)
    print()


radial net 0
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(8, 64) (8, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 1024) (64, 1024)

radial net 1
dict_keys(['Dense_0', 'Dense_1', 'Dense_2', 'Dense_3'])
(8, 64) (8, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 64) (64, 64)
cst 1.001559582187594
(64, 2560) (64, 2560)



In [15]:
from mace.tools.cg import U_matrix_real
from e3nn_jax import reduced_tensor_product_basis
from e3nn import o3
import e3nn_jax

def get_reordering(order,ir_out):
    # torch 
    irreps_in = o3.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = o3.Irreps("1x"+ir_out)
    U_matrix = U_matrix_real(irreps_in=irreps_in, irreps_out=irrep_out, correlation=order)[-1].numpy()
    U_matrix_abs = np.abs(U_matrix)

    # jax
    irreps_in = e3nn_jax.Irreps("1x0e + 1x1o + 1x2e + 1x3o")
    irrep_out = e3nn_jax.Irrep(ir_out)
    U = reduced_tensor_product_basis([irreps_in] * order, keep_ir=[irrep_out]).list[-1]
    U = np.moveaxis(U,-1,0)
    U_abs = np.abs(U)

    index = []
    signs = []
    for i in range(U.shape[-1]):
        diffs = []
        for k in range(U_matrix.shape[-1]):
            diff = np.max(np.abs(U_abs[...,i]-U_matrix_abs[...,k]))
            diffs.append(diff)
        k = np.argmin(diffs)
        diff = diffs[k]
        if diff>1.e-6:
            print(i,k,diff)
            raise ValueError("diff too large")
        
        index.append(k)
        diff = np.max(np.abs(U[...,i]-U_matrix[...,k]))
        if diff < 1.e-6:
            signs.append(1.)
        else:
            diff = np.max(np.abs(U[...,i]+U_matrix[...,k]))
            if diff < 1.e-6:
                print(i,k,"global sign differ")
                signs.append(-1.)
            else:
                raise ValueError("local sign differ")
    print(order,ir_out,len(index),U.shape[-1],len(signs))
    return np.array(index),np.array(signs)
    return np.arange(U.shape[-1])

In [16]:
ir_layers = [["0e","1o"],["0e"]]
for layer in [0,1]:
    print()
    print("layer",layer,"->",list(nn_energy_params[f"SymmetricContraction_{layer}"].keys()))

    for ir in ir_layers[layer]:
        l=int(ir[0])
        corr = len(torch_model.products[layer].symmetric_contractions.contractions[l].weights)+1
        print("   ir",ir,"l",l,"corr",corr)

        w=torch_model.products[layer].symmetric_contractions.contractions[l].weights_max.detach().cpu().numpy()

        print(f"     w{corr}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape) 
        assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"].shape[1:]

        # reorder weights
        index,signs = get_reordering(corr,ir)
        w = w[:,index,:]*signs[None,:,None]


        # last order
        skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"])
        for i,z in enumerate(species_order):
            skip_params[z,:,:] = w[i,:,:]
        nn_energy_params[f"SymmetricContraction_{layer}"][f"w{corr}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)

        print(np.abs(w).mean())

        for c in range(corr-1):
            order = corr-c-1
            w=torch_model.products[layer].symmetric_contractions.contractions[l].weights[c].detach().cpu().numpy()
            print(f"     w{order}_{ir}",w.shape,nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape)
            assert w.shape[1:] == nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"].shape[1:]

            index,signs = get_reordering(order,ir)
            w = w[:,index,:]*signs[None,:,None]

            skip_params = np.zeros_like(nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"])
            for i,z in enumerate(species_order):
                skip_params[z,:,:] = w[i,:,:]
            nn_energy_params[f"SymmetricContraction_{layer}"][f"w{order}_{ir}"] = jnp.asarray(skip_params,dtype=jnp.float64)
            print(np.abs(w).mean())



layer 0 -> ['w1_0e', 'w1_1o', 'w2_0e', 'w2_1o', 'w3_0e', 'w3_1o']
   ir 0e l 0 corr 3
     w3_0e (89, 23, 256) (120, 23, 256)


3 0e 23 23 23
0.023364469645692523
     w2_0e (89, 4, 256) (120, 4, 256)
2 0e 4 4 4
0.06560130721980657
     w1_0e (89, 1, 256) (120, 1, 256)
1 0e 1 1 1
0.12352341570609536
   ir 1o l 1 corr 3
     w3_1o (89, 51, 256) (120, 51, 256)
17 17 global sign differ
18 18 global sign differ
35 21 global sign differ
36 22 global sign differ
44 44 global sign differ
46 46 global sign differ
3 1o 51 51 51
0.012337342339413158
     w2_1o (89, 6, 256) (120, 6, 256)
2 1o 6 6 6
0.024140118511619114
     w1_1o (89, 1, 256) (120, 1, 256)
1 1o 1 1 1
0.05139102941315899

layer 1 -> ['w1_0e', 'w2_0e', 'w3_0e']
   ir 0e l 0 corr 3
     w3_0e (89, 23, 256) (120, 23, 256)
3 0e 23 23 23
0.001676299679452954
     w2_0e (89, 4, 256) (120, 4, 256)
2 0e 4 4 4
0.011771464396225765
     w1_0e (89, 1, 256) (120, 1, 256)
1 0e 1 1 1
0.05506005648868481


In [17]:
variables["params"]["nn_energy"] = nn_energy_params  
model_fennol.variables = variables 

model_fennol.save(model_name+".fnx")
model_fennol_load = FENNIX.load(model_name+".fnx")
# print("##############################")
# print("Restored model")
# for k in model_fennol_load.variables["params"]["nn_energy"].keys():
#     print(k)
#     if isinstance(nn_energy_params[k],dict):
#         for kk in nn_energy_params[k].keys():
#             print("   ",kk)
calc_fennol = FENNIXCalculator(model=model_fennol_load,atoms=atoms,use_float64=True)
atoms.calc = calc_fennol
print("MACE fennol",atoms.get_potential_energy())
atoms.calc = calc  
print("MACE torch",atoms.get_potential_energy())


MACE fennol -14.017221861330029
MACE torch -14.017221861246675


In [18]:
species = atoms.get_atomic_numbers()
coordinates = atoms.get_positions()
output = model_fennol_load(coordinates=coordinates,species=species)
print(output.keys())
print(output["nn_energy"]*au.EV)
# print(output["nn_energy_node_feats"][0])

dict_keys(['batch_index', 'coordinates', 'energy_shift', 'graph', 'natoms', 'nn_energy', 'nn_energy_node_feats', 'species'])
[-0.02513665  0.31355303  0.31355303]


In [19]:
node_feats_torch = calc.get_descriptors(atoms)
# print(node_feats_torch[0])
print(calc.get_property("node_energy",atoms=atoms))

[-0.02513665  0.31355303  0.31355303]
